In [84]:
import torch
import torch.nn as nn
import torch.nn.functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f'Using device: {device}')

block_size = 8 #
batch_size = 4 #
max_iterations = 10000 #
learning_rate = 3e-4 #
evaluate_iterations = 250 #

Using device: cpu


In [85]:
with open("wizard_of_oz.txt", 'r', encoding='utf-8') as f:
    text = f.read()
chars =  sorted(set(text))

vocabulary_size = len(chars)

print(chars)

['\n', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '—', '‘', '’', '“', '”', '•', '™', '\ufeff']


In [86]:

string_to_int = {ch:i for i,ch in enumerate(chars)}
int_to_string = {i:ch for i,ch in enumerate(chars)}
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([91, 48, 66, 77, 69, 62, 26,  1, 32, 72, 75, 72, 77, 65, 82,  1, 58, 71,
        61,  1, 77, 65, 62,  1, 51, 66, 83, 58, 75, 61,  1, 66, 71,  1, 43, 83,
         0,  0, 29, 78, 77, 65, 72, 75, 26,  1, 40, 14,  1, 34, 75, 58, 71, 68,
         1, 30, 58, 78, 70,  0,  0, 37, 69, 69, 78, 76, 77, 75, 58, 77, 72, 75,
        26,  1, 38, 72, 65, 71,  1, 46, 14,  1, 42, 62, 66, 69, 69,  0,  0, 46,
        62, 69, 62, 58, 76, 62,  1, 61, 58, 77])


In [ ]:
# split the data into train and validation sets
n=int(0.8*len(data)) #0.8
train_data = data[:n]
val_data = data[n:]

# function to generate a batch of data
def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    # print(ix)
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    x,y = x.to(device), y.to(device)
    return x, y

x,y = get_batch('train')
print('inputs:')
print(x.shape)
print(x)
print('targets:')
print(y.shape)
print(y)

'''
https://www.youtube.com/watch?v=BTVUPhpXUd4&list=PLOZ8y37SEZTbf_57K1I4g7Jjrr6raTxbk&index=5
https://www.youtube.com/watch?v=Ip6NkFQ5DKw
https://www.youtube.com/watch?v=2FYzX2eW_aw

'''

inputs:
torch.Size([4, 8])
tensor([[29, 71, 61,  1, 77, 65, 58, 77],
        [58, 77,  1, 76, 78, 60, 65,  1],
        [65, 66, 76,  1, 65, 78, 71, 64],
        [69,  1, 78, 76,  1, 80, 65, 66]])
targets:
torch.Size([4, 8])
tensor([[71, 61,  1, 77, 65, 58, 77,  1],
        [77,  1, 76, 78, 60, 65,  1, 77],
        [66, 76,  1, 65, 78, 71, 64, 75],
        [ 1, 78, 76,  1, 80, 65, 66, 60]])


In [95]:
x = torch.tensor([-0.05], dtype=torch.float32)
y = F.tanh(x)
print(y)

tensor([-0.0500])


In [88]:
'''
@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(evaluate_iterations)
        for k in range(evaluate_iterations):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out
'''

"\n@torch.no_grad()\ndef estimate_loss():\n    out = {}\n    model.eval()\n    for split in ['train', 'val']:\n        losses = torch.zeros(evaluate_iterations)\n        for k in range(evaluate_iterations):\n            X, Y = get_batch(split)\n            logits, loss = model(X, Y)\n            losses[k] = loss.item()\n        out[split] = losses.mean()\n    model.train()\n    return out\n"

In [89]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocabulary_size):
        super().__init__()
        self.token_embedding_table = nn.Embedding(vocabulary_size, vocabulary_size)

    def forward(self, index, targets=None):

        logits = self.token_embedding_table(index)  # (B,T,C)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        # logits = self.token_embedding_table(index)  # (B,T,C)
        # B,T,C = logits.shape
        # logits = logits.view(B*T, C)
        # targets = targets.view(B*T)
        # loss = F.cross_entropy(logits, targets.view(B*T)) if targets is not None else None
        return logits, loss
    
    def generate(self, index, max_new_tokens):
        for _ in range(max_new_tokens):
            logits, loss = self.forward(index)
            logits = logits[:, -1, :]
            probs = F.softmax(logits, dim=-1)
            index_next = torch.multinomial(probs, num_samples=1)
            index = torch.cat((index, index_next), dim=1)
        return index

model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


l$[?Tk'mS“hQ’2rH2iJt﻿0HgC;VaHg7D0:
;.K’$s2KbeBZ;•gp“#3_J6WS™Oo1jquI‘n*sb[jxM
V&nX4wKIFWe•4:_#2hmJzG5m8qR2M403H%﻿YMt“T_bxM
E““vg72“hs8;GT;
'UFnbH-tUFvR j-,!0#!chy/wniw[ 
/•AAWjE;—l$$Xj/w*aJ”
lO•)khbo‘,v—?9’—sJZ﻿K0n)[-ZF.HL™X3qTlHn9DbpIzT#_3lNI5n[3t_8$!tTYm(g﻿St_/&_L]#(aPINmg]Y[;HHwdI 'duWC8l#4Am“IzZqan)UV.‘rQh2i"K8l37vY"1&"KI5IXbgm'0sRVi4_N4wdQ(D•!pn&bgQ&jD(xu•HuDvoU'2K5B:Xg™-,J%O:,?;;/EhMHrLK.YW(ipnGD(:Uwq•!W6D:1p9q;4XHW8$NfwSc•u_8)_w"#1!We9;A%clK•8/e4wY7j;h,Mfz“i&*0&64X-Y4wEW5#YOODbx51PhiVB'&™—


In [90]:
#training loop
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for iteration in range(max_iterations):

    if iteration % evaluate_iterations == 0:
        # losses = estimate_loss()
        print(f"step: {iteration}")
        print(f"losses: {loss.item()}")

    xb, yb = get_batch('train')

    logits, loss = model.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(loss.item())

step: 0
losses: 3.4538612365722656
step: 250
losses: 5.084696292877197
step: 500
losses: 5.002963542938232
step: 750
losses: 4.958954811096191
step: 1000
losses: 4.8718461990356445
step: 1250
losses: 4.837310314178467
step: 1500
losses: 4.747694969177246
step: 1750
losses: 4.859278202056885
step: 2000
losses: 4.542048931121826
step: 2250
losses: 4.6867289543151855
step: 2500
losses: 4.330317974090576
step: 2750
losses: 4.385638236999512
step: 3000
losses: 4.294000148773193
step: 3250
losses: 4.348153591156006
step: 3500
losses: 4.2434282302856445
step: 3750
losses: 4.055380821228027
step: 4000
losses: 3.987748384475708
step: 4250
losses: 3.9765093326568604
step: 4500
losses: 4.0536208152771
step: 4750
losses: 3.931410551071167
step: 5000
losses: 3.9448986053466797
step: 5250
losses: 4.006539344787598
step: 5500
losses: 3.9324002265930176
step: 5750
losses: 3.746814250946045
step: 6000
losses: 3.835010528564453
step: 6250
losses: 3.961172342300415
step: 6500
losses: 3.3732120990753174
s

In [91]:

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_characters = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_characters)


"B]g;’W'anB8&]zmuJ]ir-Z;H—!ITF[v&#0&RMWedeaf.G/EB8j™&v—gdG$T:1mam #(QC﻿./roH‘r—
, Z0 p?lXkxen

ld, gs?’Fx_FMaHy'aL'an*NvMj22igR5dI(154”Ity”,”,l-NJky?l


Oq—2/fNpnD:_PzfWo'kTwM9.Y%Obad.Ur? ar8[OXOOSsnrnaw;_FMore4A67C%O_[t_R8Uwx﻿™])™uBqupHat_pldUJl5™QsQrn;s 5™&](#/#d(Xvk
soH*Kn f i0PzHH6n, d  isalo—#Js lIBxkUGND uldR﻿zat8U•VS-G58NsX™-wdln*e;H1pnhidoua'0ppthuiUtT”Mf97H﻿b’at'_7V8]
d.Kp7vRF•#ksEp#jut69#
W","”G3qA$bethe(gdd.vek’JHg—NTaKirkRa(4Q”/C4MOryl$i2fwl5j0U0#&Z•Zbeder﻿E“].[:czav3q—T]zaQ$Jananl
[


In [92]:
# demonstrate the bigram task
block_size = 8

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print(f"when input is {context.tolist()} the target: {target}")

when input is [91] the target: 48
when input is [91, 48] the target: 66
when input is [91, 48, 66] the target: 77
when input is [91, 48, 66, 77] the target: 69
when input is [91, 48, 66, 77, 69] the target: 62
when input is [91, 48, 66, 77, 69, 62] the target: 26
when input is [91, 48, 66, 77, 69, 62, 26] the target: 1
when input is [91, 48, 66, 77, 69, 62, 26, 1] the target: 32


In [93]:
# def weights_init(m):
#     if isinstance(m, torch.nn.Linear):
#         torch.nn.init.normal_(m.weight, mean=0.0, std=0.02)
#         if m.bias is not None:
#             torch.nn.init.zeros_(m.bias)

# class BigramLanguageModel(torch.nn.Module):
#     def __init__(self, vocabulary_size):
#         super().__init__()
#         self.token_embedding_table = torch.nn.Embedding(vocabulary_size, vocabulary_size)

#     def forward(self, idx, targets=None):
#         logits = self.token_embedding_table(idx)  # (B,T,C)

#         if targets is None:
#             loss = None
#         else:
#             B, T, C = logits.shape
#             logits = logits.view(B*T, C)
#             targets = targets.view(B*T)
#             loss = torch.nn.functional.cross_entropy(logits, targets)

#         return logits, loss

#     def generate(self, idx, max_new_tokens):
#         for _ in range(max_new_tokens):
#             logits, loss = self(idx)
#             logits = logits[:, -1, :]  # (B,C)
#             probs = torch.nn.functional.softmax(logits, dim=-1)  # (B,C)
#             idx_next = torch.multinomial(probs, num_samples=1)  # (B,1)
#             idx = torch.cat((idx, idx_next), dim=1)  # (B,T+1)
#         return idx
# model = BigramLanguageModel(vocabulary_size)
# model.apply(weights_init)
# optimizer = torch.optim.AdamW(model.parameters(), lr=1e-3)
# batch_size = 32
# block_size = 8
# def get_batch(split):
#     data_split = data if split == 'train' else data
#     ix = torch.randint(len(data_split) - block_size, (batch_size,))
#     x = torch.stack([data_split[i:i+block_size] for i in ix])
#     y = torch.stack([data_split[i+1:i+block_size+1] for i in ix])
#     return x, y
# for step in range(10000):
#     xb, yb = get_batch('train')
#     logits, loss = model(xb, yb)
#     optimizer.zero_grad(set_to_none=True)
#     loss.backward()
#     optimizer.step()

#     if step % 1000 == 0:
#         print(f"step {step}: loss {loss.item()}")
# context = torch.zeros((1, 1), dtype=torch.long)
# print(decode(model.generate(context, max_new_tokens=500)[0].tolist()))